<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/21_iSDAsoil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract Effective Cation Exchange Capacity from iSDAsoil platform


## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=cfz-KwRv7KRftJ_s73lb9O9ZASLQT8Iw6lJRGkITW8Q&tc=gMnW5-Z7XIOPUbFzKf1HM3uQDgVmsSalq9i8KUeeIHk&cc=E-9Is5Vafot5DBS6R0VsX_bHnWAjO76sOh_pIPSYQjA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BWQodGzc2GHxFrIei8DPUgayUkRuMB7tdRo-HPvWb0uHR7TuEaq65k

Successfully saved authorization token.


## Input arguments for data extraction

In [ ]:
# Area of interest
# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
# aoi_name = "MKR_NS_buff_5km"

# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_PACE")
# aoi_name = "MKR_PACE"

aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR")
aoi_name = "MKR"

# GEE layer ID
layer_name = "RS_069"

## Extract



In [ ]:
# Pixel values must be back-transformed with exp(x/10)-1.

cat_exchange = ee.Image("ISDASOIL/Africa/v1/cation_exchange_capacity") \
              .select(["mean_0_20", "mean_20_50"]) \
              .divide(10).exp().subtract(1) \
              .reduce(ee.Reducer.mean()) \
              .clip(aoi)

In [ ]:
cat_exchange.projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:3857',
 'transform': [30, 0, -3502582.8076430312, 0, -30, 4530073.13045243]}

## Export data - create task

`filenamePrefix` should be in format: place_layer_timeperiod

In [ ]:
task1 = ee.batch.Export.image.toDrive(image = cat_exchange,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = layer_name + "_" + aoi_name,
                                     scale = 30,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
task1.start()

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
task_id = task1.status()['id']
task_state = task1.status()['state']
print(task_id, task_state)

VIIS7PMMMAGKUO2FYUEWIB7G RUNNING
